In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import math
import random

Get templates

In [2]:
def get_templates():
    templates = {}
    for x in os.listdir("small_templates"):
        templates[x] = cv2.imread("small_templates/" + x)
    
    template_labels = []   
    c=0
    for t in templates:
        templates[t] = cv2.cvtColor(templates[t], cv2.COLOR_BGR2GRAY) # convert to grey
        templates[t] = cv2.medianBlur(templates[t],5) # blur
        templates[t] = cv2.Canny(templates[t], 50, 200, apertureSize = 3) # edge detection
        
        template_labels.append((c,templates[t]))
        c+=1
    return template_labels
        
        
#         cv2.imshow("labels", template_labels[c][1])
#         cv2.waitKey(0)

#         print(template_labels[c][0])

#         cv2.imshow("templates", templates[t])
#         cv2.waitKey(0)
        

In [3]:
template_labels = get_templates()

Get images

In [4]:
def get_images():
    raw = {}
    for x in os.listdir("raw"):
        raw[x] = cv2.imread("raw/" + x)

    images = raw.copy()
    image_labels = []   
    c=0
    for i in images:
#         images[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY) # convert to grey
#         images[i] = cv2.medianBlur(images[i],5) # blur
#         images[i] = cv2.Canny(images[i], 50, 200, apertureSize = 3) # edge detection
        
        image_labels.append((c,images[i]))
        c+=1
#         cv2.imshow("images", images[i])
#         cv2.waitKey(0)
        
    return image_labels

In [5]:
image_labels = get_images()

Skew images

In [24]:
def skew_images(raw):
    filtered = [item[1] for item in raw]
    filtered_labels = [item[0] for item in raw]
    print(filtered_labels)
    c=0
    data = []
    for photo in filtered:
        for x in range(5):
            new = photo.copy()
            row, col, _ = new.shape

            kvalue = random.randint(1,6)
            ksize = (kvalue,kvalue)
            blurred = cv2.blur(new,ksize)

            number_of_pixels = random.randint(300, 10000)
            for i in range(number_of_pixels):

                y_coord=random.randint(0, row - 1)

                x_coord=random.randint(0, col - 1)

                blurred[y_coord][x_coord] = 255

            number_of_pixels = random.randint(300 , 10000)
            for i in range(number_of_pixels):

                y_coord=random.randint(0, row - 1)

                x_coord=random.randint(0, col - 1)

                blurred[y_coord][x_coord] = 0
                
            data.append((c,blurred))
            
        c+=1
        
#     for photo in data:
#         cv2.imshow("skewed", photo[1])
#         cv2.waitKey(0)
        
    return data
        
    



# median1 = cv2.medianBlur(filtered,5)

# # edge detection
# edges1 = cv2.Canny(median, 50, 200, apertureSize = 3)



# cv2.imshow("Template", blurred)
# cv2.waitKey(0)

In [25]:
data = skew_images(image_labels)

[0, 1, 2]


In [39]:
def filter_images(data):
    dataset = []
    for photo in data:
        temp = cv2.medianBlur(photo[1],5) # blur
        temp = cv2.Canny(temp, 50, 200, apertureSize = 3) # edge detection
        dataset.append((photo[0],temp))
        
#     for photo in dataset:
#         cv2.imshow("test", photo[1])
#         cv2.waitKey(0)

    print([x[0] for x in dataset])
        
    return dataset
    

In [40]:
dataset = filter_images(data)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]


In [69]:
def match(photo, template):
    img = photo[1]
    img_val = photo[0]
    temp = template[1]
    temp_val = template[0]
    
    result = cv2.matchTemplate(img, temp, cv2.TM_CCOEFF_NORMED)
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(result)

#     (startX, startY) = maxLoc
#     endX = startX + temp.shape[1]
#     endY = startY + temp.shape[0]

#     cv2.rectangle(new, (startX, startY), (endX, endY), (255, 0, 0), 3)

    threshold = 0.3
    flag = False
    if np.amax(result) > threshold:
        flag = True
        
#     print(flag)
#     print("Img Val: "+str(img_val))
#     print("Temp Val: "+str(temp_val))
    
    return flag, img_val, temp_val

#     cv2.imshow("Output", new)
#     cv2.waitKey(0)

#     return flag

In [70]:
correct = 0
should_be_true = 0
wrong = 0
total = 0
for photo in dataset:
    for t in template_labels:
        flag, img_val, temp_val = match(photo,t)
        if flag == True and img_val == temp_val:
            correct+=1
        elif flag == False and img_val == temp_val:
            should_be_true+=1
        elif flag == True and img_val != temp_val:
            wrong+=1
        total+=1
        print(flag)
        
print("Correct: "+str(correct))
print("Should be correct: "+str(should_be_true))
print("Wrong: "+str(wrong))
print("Total: "+str(total))

True
False
False
True
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
True
False
False
True
False
False
False
True
False
False
True
False
False
True
False
False
True
False
False
True
Correct: 15
Should be correct: 0
Wrong: 0
Total: 45
